In [25]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [26]:
# Load the datasets
awards_players_df = pd.read_csv('../dataset/awards_players.csv')
coaches_df = pd.read_csv('../dataset/coaches.csv')
players_df = pd.read_csv('../dataset/players.csv')
players_teams_df = pd.read_csv('../dataset/players_teams.csv')
series_post_df = pd.read_csv('../dataset/series_post.csv')
teams_df = pd.read_csv('../dataset/teams.csv')
teams_post_df = pd.read_csv('../dataset/teams_post.csv')

In [27]:
# Remove useless columns from the datasets
awards_players_df = awards_players_df.drop(columns=['lgID'])
players_df = players_df.drop(columns=['firstseason', 'lastseason', 'deathDate'])
coaches_df = coaches_df.drop(columns=['lgID'])
series_post_df = series_post_df.drop(columns=['lgIDLoser', 'lgIDWinner'])
teams_post_df = teams_post_df.drop(columns=['lgID'])
teams_df = teams_df.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])
players_teams_df = players_teams_df.drop(columns=['lgID'])

In [28]:
# Display the first few rows of each cleaned DataFrame (optional)
awards_players_df.head()

,playerID,award,year
0,thompti01w,All-Star Game Most Valuable Player,1
1,leslili01w,All-Star Game Most Valuable Player,2
2,leslili01w,All-Star Game Most Valuable Player,3
3,teaslni01w,All-Star Game Most Valuable Player,4
4,swoopsh01w,All-Star Game Most Valuable Player,6


In [29]:
players_df.head()

,bioID,pos,height,weight,college,collegeOther,birthDate
0,abrahta01w,C,74.0,190,George Washington,NaN,1975-09-27
1,abrossv01w,F,74.0,169,Connecticut,NaN,1980-07-09
2,adairje01w,C,76.0,197,George Washington,NaN,1986-12-19
3,adamsda01w,F-C,73.0,239,Texas A&M,Jefferson College (JC),1989-02-19
4,adamsjo01w,C,75.0,180,New Mexico,NaN,1981-05-24


In [30]:
coaches_df.head()

,coachID,year,tmID,stint,won,lost,post_wins,post_losses
0,adamsmi01w,5,WAS,0,17,17,1,2
1,adubari99w,1,NYL,0,20,12,4,3
2,adubari99w,2,NYL,0,21,11,3,3
3,adubari99w,3,NYL,0,18,14,4,4
4,adubari99w,4,NYL,0,16,18,0,0


In [31]:
series_post_df.head()

,year,round,series,tmIDWinner,tmIDLoser,W,L
0,1,FR,A,CLE,ORL,2,1
1,1,FR,B,NYL,WAS,2,0
2,1,FR,C,LAS,PHO,2,0
3,1,FR,D,HOU,SAC,2,0
4,1,CF,E,HOU,LAS,2,0


In [32]:
teams_post_df.head()

,year,tmID,W,L
0,1,HOU,6,0
1,1,ORL,1,2
2,1,CLE,3,3
3,1,WAS,0,2
4,1,NYL,4,3


In [33]:
teams_df.head()

,year,tmID,franchID,confID,rank,playoff,firstRound,semis,finals,name,...,GP,homeW,homeL,awayW,awayL,confW,confL,min,attend,arena
0,9,ATL,ATL,EA,7,N,NaN,NaN,NaN,Atlanta Dream,...,34,1,16,3,14,2,18,6825,141379,Philips Arena
1,10,ATL,ATL,EA,2,Y,L,NaN,NaN,Atlanta Dream,...,34,12,5,6,11,10,12,6950,120737,Philips Arena
2,1,CHA,CHA,EA,8,N,NaN,NaN,NaN,Charlotte Sting,...,32,5,11,3,13,5,16,6475,90963,Charlotte Coliseum
3,2,CHA,CHA,EA,4,Y,W,W,L,Charlotte Sting,...,32,11,5,7,9,15,6,6500,105525,Charlotte Coliseum
4,3,CHA,CHA,EA,2,Y,L,NaN,NaN,Charlotte Sting,...,32,11,5,7,9,12,9,6450,106670,Charlotte Coliseum


In [34]:
players_teams_df.head()

,playerID,year,stint,tmID,GP,GS,minutes,points,oRebounds,dRebounds,...,PostBlocks,PostTurnovers,PostPF,PostfgAttempted,PostfgMade,PostftAttempted,PostftMade,PostthreeAttempted,PostthreeMade,PostDQ
0,abrossv01w,2,0,MIN,26,23,846,343,43,131,...,0,0,0,0,0,0,0,0,0,0
1,abrossv01w,3,0,MIN,27,27,805,314,45,101,...,0,0,0,0,0,0,0,0,0,0
2,abrossv01w,4,0,MIN,30,25,792,318,44,97,...,1,8,8,22,6,8,8,7,3,0
3,abrossv01w,5,0,MIN,22,11,462,146,17,57,...,2,3,7,23,8,4,2,8,2,0
4,abrossv01w,6,0,MIN,31,31,777,304,29,78,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Merge datasets
players_teams_merged = pd.merge(players_df, players_teams_df, left_on='bioID', right_on='playerID')
players_teams_awards = pd.merge(players_teams_merged, awards_players_df, on=['year', 'playerID'], how='left')

teams_plus_post = pd.merge(teams_df, teams_post_df, on=['year', 'tmID'], how='left')

teams_coaches = pd.merge(teams_plus_post, coaches_df, on=['year', 'tmID'], how='left')
teams_coaches.rename(columns={
    'won_x': 'won_team',
    'lost_x': 'lost_team',
    'won_y': 'won_coach',
    'lost_y': 'lost_coach'
}, inplace=True)

final = pd.merge(teams_coaches, players_teams_awards, on=['year', 'tmID'], how='left')

award_scores = {
    'All-Star Game Most Valuable Player': 7,
    'Coach of the Year': 10,
    'Defensive Player of the Year': 7,
    'Kim Perrot Sportsmanship Award': 0,
    'Most Improved Player': 5,
    'Most Valuable Player': 10,
    'Rookie of the Year': 5,
    'Sixth Woman of the Year': 6,
    'WNBA Finals Most Valuable Player': 8,
    'WNBA All-Decade Team': 6,
    'WNBA All Decade Team Honorable Mention': 4
}

def shift_statistics_for_prediction(data):
    # Ensure data is sorted by team ID and year
    data = data.sort_values(by=['tmID', 'year'])

    # Shift statistics to align the previous year's features with the current year's outcomes
    shifted_data = data.copy()
    for col in data.columns:
        if col not in ['year', 'tmID', 'playoff']:
            shifted_data[f'prev_{col}'] = data.groupby('tmID')[col].shift(1)

    # Label column remains from the current season
    shifted_data = shifted_data.dropna(subset=[f'prev_{col}' for col in data.columns if col not in ['year', 'tmID', 'playoff']])

    return shifted_data

# Map the awards to their scores
final['award_score'] = final['award'].map(award_scores).fillna(0)

# Group by year, team, and player, summing award scores for each unique combination
award_points = final.groupby(['year', 'tmID', 'bioID']).agg(
    total_award_score=pd.NamedAgg(column='award_score', aggfunc='sum')
).reset_index()

# Now, `award_points` has one row per year, tmID, and bioID combination, with the summed award scores

# Merge this aggregated award score back into the main `final` dataframe
final = pd.merge(
    final.drop(columns=['award', 'award_score']),  # Drop award columns to avoid duplicates in merge
    award_points,
    on=['year', 'tmID', 'bioID'],
    how='left'
)

# Fill any NaNs in `total_award_score` with 0, in case some players have no awards
final['total_award_score'] = final['total_award_score'].fillna(0)

# Save intermediate file if needed
final.to_csv('final.csv', index=False)

player_points = final.groupby(['year', 'tmID', 'bioID']).agg(
    total_points=pd.NamedAgg(column='points', aggfunc='sum')
).reset_index()

# Columns to aggregate (already present in the final dataframe)
columns_to_aggregate = [
    'o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_oreb', 'o_dreb', 'o_reb',
    'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta',
    'd_3pm', 'd_3pa', 'd_oreb', 'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk',
    'd_pts', 'won_team', 'lost_team', 'GP_x', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'playoff'
]

# Grouping the data and aggregating using the columns from 'final'
grouped_data = final.groupby(['year', 'tmID']).agg(
    {col: 'first' for col in columns_to_aggregate}
).reset_index()

# Function to retrieve and sort players by points
def sorted_players(year, tmID):
    players = player_points[(player_points['year'] == year) & (player_points['tmID'] == tmID)]
    return list(players.sort_values(by='total_points', ascending=False)['bioID'])

# Add ranked player columns dynamically
def expand_players_list(row):
    players = sorted_players(row['year'], row['tmID'])
    player_dict = {f'ranked_player_{i+1}': players[i] if i < len(players) else None for i in range(13)}
    return pd.Series(player_dict)

# Apply expansion of player lists
expanded_players = grouped_data.apply(expand_players_list, axis=1)
grouped_data_with_players = pd.concat([grouped_data, expanded_players], axis=1)

# Map playoff column to binary
grouped_data_with_players['playoff'] = grouped_data_with_players['playoff'].map({'Y': 1, 'N': 0})

# Sort and create playoffNextYear column
grouped_data_with_players = grouped_data_with_players.sort_values(['tmID', 'year'])
grouped_data_with_players['playoffNextYear'] = grouped_data_with_players['playoff'].shift(-1)
grouped_data_with_players.loc[grouped_data_with_players['tmID'] != grouped_data_with_players['tmID'].shift(-1), 'playoffNextYear'] = None
grouped_data_with_players['playoffNextYear'] = grouped_data_with_players['playoffNextYear'].fillna(0)


In [36]:
# Apply the function to shift statistics
shifted_data = shift_statistics_for_prediction(grouped_data_with_players)

# Features and label
features = [f'prev_{col}' for col in ['o_pts', 'won_team', 'o_stl']]  # Use shifted columns
label = 'playoffNextYear'

# Train-test split
train_data = shifted_data[shifted_data['year'] <= 5]  # Adjust year threshold as needed
test_data = shifted_data[shifted_data['year'].isin([6, 7])]  # Adjust for test years

# Separate features and labels
x_train, y_train = train_data[features], train_data[label]
x_test, y_test = test_data[features], test_data[label]

# Train the model
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

# Make predictions
test_data = test_data.copy()  # Create a deep copy to avoid warnings
test_data['proba_class_1'] = model.predict_proba(x_test)[:, 1]
test_data['pred'] = model.predict(x_test)

# Evaluate model accuracy
print("Accuracy:", accuracy_score(y_test, test_data['pred']))
print("Classification Report:\n", classification_report(y_test, test_data['pred']))

# Display the first few rows of test_data to see probabilities and predictions
print(test_data[['proba_class_1', 'pred']])


Accuracy: 0.7333333333333333
Classification Report:
               precision    recall  f1-score   support

         0.0       0.75      0.75      0.75         8
         1.0       0.71      0.71      0.71         7

    accuracy                           0.73        15
   macro avg       0.73      0.73      0.73        15
weighted avg       0.73      0.73      0.73        15

     proba_class_1  pred
75             0.0   0.0
88             0.0   0.0
77             1.0   1.0
91             0.0   0.0
78             1.0   1.0
92             0.0   0.0
80             1.0   1.0
94             0.0   0.0
95             0.0   0.0
96             0.0   0.0
83             1.0   1.0
97             1.0   1.0
85             1.0   1.0
100            1.0   1.0
101            0.0   0.0
